Construction des cartes au format html pour la visualisation des bâtiments BNB et Unofi
-> utilisés dans la démo de novembre 2022

In [2]:
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import sys
pd.set_option('display.max_columns', None)
import json
import geopy.distance

df_unofi=pd.read_csv("C:/Users/FilipeAfonso/Documents/prez__ia/f2_demo.csv",sep=";")
df_ub=pd.read_csv("C:/Users/FilipeAfonso/Documents/map/map/unofi_bnb.csv",sep=";")
# df_unofi.columns

In [3]:
dept=75
df=gpd.read_file("C:/Users/FilipeAfonso/Documents/map/map/data/bnb-export/bnb-export-"+str(dept)+".gpkg/bnb_export_"+str(dept)+".gpkg")


In [4]:
df_batiment=df

variables =['adedpe202006_logtype_mur_ep_mat_ext','adedpe202006_logtype_mur_mat_ext','adedpe202006_logtype_mur_pos_isol_ext','adedpe202006_logtype_mur_u_ext',
'adedpe202006_logtype_nom_methode_dpe','adedpe202006_logtype_pb_mat','adedpe202006_logtype_pb_pos_isol','adedpe202006_logtype_pb_type_adjacence','adedpe202006_logtype_pb_u', 
'adedpe202006_logtype_perc_surf_vitree_ext','adedpe202006_logtype_periode_construction','adedpe202006_logtype_ph_mat','adedpe202006_logtype_ph_pos_isol', 
'adedpe202006_logtype_ph_type_adjacence','cerffo2020_annee_construction','cerffo2020_mat_mur_txt','cerffo2020_mat_toit_txt']

df_batiment=df_batiment[['bnb_id','etaban202111_id' ,'insee_nom_commune','geometry']+variables]
df_batiment['noyau']=df_batiment['geometry'].centroid.to_crs(epsg=4326)
df_batiment['geometry'] = df_batiment['geometry'].to_crs(epsg=4326)
print(df_batiment['noyau'].head())

c:\Users\FilipeAfonso\AppData\Local\Programs\Python\Python310\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


0    POINT (2.33507 48.86134)
1    POINT (2.34113 48.86061)
2    POINT (2.34141 48.86047)
3    POINT (2.34153 48.86052)
4    POINT (2.34151 48.86041)
Name: noyau, dtype: geometry


In [5]:
df_batiment=df_batiment[df_batiment[variables].isnull().all(axis=1)==False]
df_batiment.head()

,bnb_id,etaban202111_id,insee_nom_commune,geometry,adedpe202006_logtype_mur_ep_mat_ext,adedpe202006_logtype_mur_mat_ext,adedpe202006_logtype_mur_pos_isol_ext,adedpe202006_logtype_mur_u_ext,adedpe202006_logtype_nom_methode_dpe,adedpe202006_logtype_pb_mat,adedpe202006_logtype_pb_pos_isol,adedpe202006_logtype_pb_type_adjacence,adedpe202006_logtype_pb_u,adedpe202006_logtype_perc_surf_vitree_ext,adedpe202006_logtype_periode_construction,adedpe202006_logtype_ph_mat,adedpe202006_logtype_ph_pos_isol,adedpe202006_logtype_ph_type_adjacence,cerffo2020_annee_construction,cerffo2020_mat_mur_txt,cerffo2020_mat_toit_txt,noyau
0,75101000AF0002_58a54d854571b71,75101_8249_00107,PARIS-1ER-ARRONDISSEMENT,"MULTIPOLYGON (((2.33699 48.86133, 2.33698 48.8...",None,None,None,NaN,None,None,None,None,NaN,NaN,None,None,None,None,1750.0,BRIQUES - PIERRE,ARDOISES - ZINC ALUMINIUM,POINT (2.33507 48.86134)
2,75101000AK0003_932627cd02035eb,75101_8249_00089,PARIS-1ER-ARRONDISSEMENT,"MULTIPOLYGON (((2.34136 48.86044, 2.34137 48.8...",None,None,None,NaN,None,None,None,None,NaN,NaN,None,None,None,None,1850.0,BRIQUES - MEULIERE,ARDOISES - ZINC ALUMINIUM,POINT (2.34141 48.86047)
3,75101000AK0004_16c39335599af75,75101_8249_00087,PARIS-1ER-ARRONDISSEMENT,"MULTIPOLYGON (((2.34148 48.86058, 2.34161 48.8...",None,None,None,NaN,None,None,None,None,NaN,NaN,None,None,None,None,1850.0,PIERRE,ZINC ALUMINIUM,POINT (2.34153 48.86052)
4,75101000AK0005_f53245cccf4c5be,75101_7279_00005,PARIS-1ER-ARRONDISSEMENT,"MULTIPOLYGON (((2.34156 48.86047, 2.34156 48.8...",None,murs en pierre de taille et moellons avec remp...,ITI,NaN,DPE vierge,plancher entre solives bois avec ou sans rempl...,None,None,NaN,NaN,<1948,combles aménagés sous rampant,ITI,None,1808.0,PIERRE,ZINC ALUMINIUM,POINT (2.34151 48.86041)
5,75101000AK0006_e6f5425331b29a0,75101_8249_00085,PARIS-1ER-ARRONDISSEMENT,"MULTIPOLYGON (((2.34156 48.86042, 2.34156 48.8...",None,murs en pierre de taille et moellons constitué...,ITI,NaN,DPE vierge,None,None,None,NaN,NaN,<1948,None,None,None,2002.0,BOIS - PIERRE,ZINC ALUMINIUM,POINT (2.34167 48.86044)


Fonction pour l'affichage sur une même carte des polygones représentant les batiments de la BNB et d'UNOFI
enregistrement au format html

In [6]:
def build_map(row,bat_actif,df_batiment):

    bat_actif = bat_actif.to_crs(epsg=4326)
    xactif=float(bat_actif['geometry'].centroid.x)
    yactif=float(bat_actif['geometry'].centroid.y)
    print(xactif, yactif)
    # m = folium.Map(location=(row['form_LAT'],row['form_LNG']), zoom_start=20, tiles='CartoDB positron')
    m = folium.Map(location=(yactif,xactif), zoom_start=20, tiles='CartoDB positron')
    
    # # Use WGS 84 (epsg:4326) as the geographic coordinate system
    count=0
    df_batiment = df_batiment.to_crs(epsg=4326)
    for _, r in df_batiment.iterrows():
        try:
            xbat=float(r['geometry'].centroid.x)
            ybat=float(r['geometry'].centroid.y)
            # print(xbat,ybat)
            if(xbat>(xactif-0.015) and xbat<(xactif+0.015) and ybat>(yactif-0.015) and ybat<(yactif+0.015)):
                # Without simplifying the representation of each borough,
                # # the map might not be displayed
                sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.000001)
                geo_j = sim_geo.to_json()
                geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'orange', 'color': 'orange'})
                folium.Popup(r['bnb_id']).add_to(geo_j)
                geo_j.add_to(m)
                # count=count+1
                # if(count>10): break
   
        except:
            continue

    for _, r in bat_actif.iterrows():
        try:
            # # Without simplifying the representation of each borough,
            # # # the map might not be displayed
            sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.000001)
            geo_j = sim_geo.to_json()
            geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'green','color':'green'},)
            html=f"""
            <h5>{'Identifiant Stonal : ' + row['code']+" : "+eval(row['assetData.ADDRESS'])[0].get('value')}</h5>
            <h5> {"identifiant BNB : "+r['bnb_id']}</h5>
            """
            html2=f"""
            <h5>{'Identifiant Stonal : ' + row['code']+" : "+eval(row['assetData.ADDRESS'])[0].get('value')}</h5>
            <h5> {"identifiant BNB : "+r['bnb_id']}</h5>
            <h5> {"Geometry BNB : "+str(r['geometry'])}</h5>
            """
            # <li>{r['adedpe202006_logtype_classe_conso_ener']}<li>
            # <li>{r['adedpe202006_logtype_classe_estim_ges']}<li>
            # <li>{r['adedpe202006_logtype_conso_ener']}<li>
            # <li>{r['adedpe202006_logtype_conso_ener']}<li>
            # <li>{r['adedpe202006_logtype_inertie']}<li>
            # <li>{r['adedpe202006_logtype_mur_ep_mat_ext']}<li>
            # <li>{r['adedpe202006_logtype_mur_mat_ext']}<li>
            # <li>{r['adedpe202006_logtype_mur_pos_isol_ext']}<li>
            # Find the columns where each value is null

            # cols_vides = [col for col in r.columns if r[col].isnull().all()]
            # r.drop(cols_vides,axis=1,inplace=True)
            sortie = open('C:/Users/FilipeAfonso/Documents/ESG/text'+row['code']+'.html', "w")
            print(html2,file=sortie)
            sortie.close()
            iframe = folium.IFrame(html=html, width=600, height=100)
            popup = folium.Popup(iframe, max_width=600)
            popup.add_to(geo_j)
            # folium.Popup(r['bnb_id']).add_to(geo_j)

            folium.Marker((float(bat_actif['geometry'].centroid.y),float(bat_actif['geometry'].centroid.x)), popup=row['code'],icon=folium.Icon(color='green')).add_to(geo_j)
            geo_j.add_to(m)
            m.location=(float(bat_actif['geometry'].centroid.y),float(bat_actif['geometry'].centroid.x))
        except:
            # sys.exit(0)
            continue   

    m.save('C:/Users/FilipeAfonso/Documents/ESG/map'+row['code']+'.html')
    # m

    return


In [7]:
def build_map_new(row,bat_actif,df_batiment,jeu):

    xactif=float(bat_actif['noyau'].x)
    yactif=float(bat_actif['noyau'].y)
    print(xactif, yactif)
    # m = folium.Map(location=(row['form_LAT'],row['form_LNG']), zoom_start=20, tiles='CartoDB positron')
    m = folium.Map(location=(yactif,xactif), zoom_start=20, tiles='CartoDB positron')
    
    count=0
    for _, r in df_batiment.iterrows():
        try:
            if (geopy.distance.geodesic((yactif,xactif), (r['noyau'].y,r['noyau'].x)).km<jeu):
                # Without simplifying the representation of each borough,
                # # the map might not be displayed
                sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.000001)
                geo_j = sim_geo.to_json()
                geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'orange', 'color': 'orange'})
                folium.Popup(r['bnb_id']).add_to(geo_j)
                geo_j.add_to(m)
                # count=count+1
                # if(count>10): break
   
        except:
            continue

    for _, r in bat_actif.iterrows():
        try:
            # # Without simplifying the representation of each borough,
            # # # the map might not be displayed
            sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.000001)
            geo_j = sim_geo.to_json()
            geo_j = folium.GeoJson(data=geo_j,style_function=lambda x: {'fillColor': 'green','color':'green'})
            html=f"""
            <h5>{'Identifiant Stonal : ' + row['code']+" : "+eval(row['assetData.ADDRESS'])[0].get('value')}</h5>
            <h5> {"identifiant BNB : "+r['bnb_id']}</h5>
            """
            html2=f"""
            <h5>{'Identifiant Stonal : ' + row['code']+" : "+eval(row['assetData.ADDRESS'])[0].get('value')}</h5>
            <h5> {"identifiant BNB : "+r['bnb_id']}</h5>
            <h5> {"Geometry BNB : "+str(r['geometry'])}</h5>
            """
            # <li>{r['adedpe202006_logtype_classe_conso_ener']}<li>
            # <li>{r['adedpe202006_logtype_classe_estim_ges']}<li>
            # <li>{r['adedpe202006_logtype_conso_ener']}<li>
            # <li>{r['adedpe202006_logtype_conso_ener']}<li>
            # <li>{r['adedpe202006_logtype_inertie']}<li>
            # <li>{r['adedpe202006_logtype_mur_ep_mat_ext']}<li>
            # <li>{r['adedpe202006_logtype_mur_mat_ext']}<li>
            # <li>{r['adedpe202006_logtype_mur_pos_isol_ext']}<li>
            # Find the columns where each value is null

            # cols_vides = [col for col in r.columns if r[col].isnull().all()]
            # r.drop(cols_vides,axis=1,inplace=True)
            sortie = open('C:/Users/FilipeAfonso/Documents/ESG/text'+row['code']+'.html', "w")
            print(html2,file=sortie)
            sortie.close()
            iframe = folium.IFrame(html=html, width=600, height=100)
            popup = folium.Popup(iframe, max_width=600)
            popup.add_to(geo_j)
            # folium.Popup(r['bnb_id']).add_to(geo_j)

            folium.Marker((yactif,xactif), popup=row['code'],icon=folium.Icon(color='green')).add_to(geo_j)
            geo_j.add_to(m)
            m.location=((yactif,xactif))
        except:
            # sys.exit(0)
            continue   

    m.save('C:/Users/FilipeAfonso/Documents/ESG/map'+row['code']+'.html')
    # m

    return


Traitement bâtiment par bâtiment UNOFI

In [8]:
# print(df_batiment.columns)
jeu=0.015
for index,row in df_unofi.iterrows():
    ub_actif=df_ub[df_ub['id_unofi']==row['code']]
    if(len(ub_actif)>0 and str(eval(row['form_ADRESSE'])['departement']['numero'])==str(dept)):
        commune=str(eval(row['form_ADRESSE'])['ville']['nom'])
        print(commune)
        bat_actif=df_batiment[df_batiment['etaban202111_id']==ub_actif.iloc[0][1]]
        if(len(bat_actif)>0) :
            # build_map(row,bat_actif,df_batiment[df_batiment['insee_nom_commune']==commune])
            # build_map(row,bat_actif,df_batiment)
            # build_map_new(row,bat_actif, df_batiment[df_batiment['insee_nom_commune']==commune],jeu)
            # build_map_light(row,bat_actif, df_batiment,jeu)
            m=build_map_test(row,bat_actif, df_batiment,jeu)
            listeok = open('C:/Users/FilipeAfonso/Documents/ESG/liste_unofi.csv', "a")
            print(row['code']+";"+eval(row['assetData.ADDRESS'])[0].get('value'),file=listeok)
            listeok.close()
            # sys.exit(0)



PARIS
PARIS


NameError: name 'build_map_test' is not defined

In [ ]:
def build_map_light(bat_actif,df_batiment,jeu):



    xactif=float(bat_actif.iloc[0].noyau.x)
    yactif=float(bat_actif.iloc[0].noyau.y)
    print(xactif, yactif)
    m = folium.Map(location=(yactif,xactif), zoom_start=20, tiles='CartoDB positron')


    df_batiment=df_batiment[['bnb_id','geometry','noyau','cerffo2020_mat_toit_txt']]
    df_batiment=df_batiment[df_batiment.isnull().any(axis=1)==False]
    df_batiment=df_batiment[(df_batiment.noyau.x.between(xactif-jeu, xactif+jeu)) & (df_batiment.noyau.y.between(yactif-jeu, yactif+jeu))]

    # Without simplifying the representation of each geometry,the map might not be displayed
    # sim_geo = gpd.GeoSeries(df_batiment['geometry']).simplify(tolerance=0.000001)
    #geo_j = sim_geo.to_json()

    geo_j = df_batiment.drop(['noyau'], axis=1).to_json()

    geo_style_function = lambda x: {'color' :  'gold','opacity' : 0.50,'weight' : 2,}
    geo_highlight_function = lambda x: {'color': 'red', 'opacity' : 0.9,'weight': 4,'dashArray' : '3, 6'}

    # main display function using the two previous functions
    geo = folium.features.GeoJson(
    data=geo_j,
    name = 'geo',
    control = True,
    style_function = geo_style_function, 
    highlight_function = geo_highlight_function,

    # the tooltip is where the info display happens
    # using "folium.features.GeoJsonTooltip" function instead of basic text tooltip
            tooltip=folium.features.GeoJsonTooltip(
                fields=[
                    'bnb_id',
                    'cerffo2020_mat_toit_txt', 
                ],
                aliases=[
                    "Identifiant: ",
                    "Toiture: ",
                ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
                )
    )
    # m.add_child(geo)
    geo.add_to(m)

    for p in bat_actif.noyau : folium.Marker((p.y,p.x),icon=folium.Icon(color='green')).add_to(geo)


    bat_actif=bat_actif[['bnb_id','geometry','cerffo2020_mat_toit_txt']]
    # bat_actif=bat_actif[bat_actif.isnull().any(axis=1)==False]

    # Without simplifying the representation of each geometry,the map might not be displayed
    # sim_geo = gpd.GeoSeries(df_batiment['geometry']).simplify(tolerance=0.000001)
    #geo_j = sim_geo.to_json()

    geo_j = bat_actif.to_json()



    geo_style_function = lambda x: {'color' :  'green','opacity' : 0.50,'weight' : 2,}
    geo_highlight_function = lambda x: { 'color': 'black','opacity' : 0.9,'weight': 4,'dashArray' : '3, 6' }

    # main display function using the two previous functions
    geo = folium.features.GeoJson(
    data=geo_j,
    name = 'geo_unofi',
    control = True,
    style_function = geo_style_function, 
    highlight_function = geo_highlight_function,

    # the tooltip is where the info display happens
    # using "folium.features.GeoJsonTooltip" function instead of basic text tooltip
            tooltip=folium.features.GeoJsonTooltip(
                fields=[
                    'bnb_id',
                    'cerffo2020_mat_toit_txt', 
                ],
                aliases=[
                    "Identifiant: ",
                    "Toiture: ",
                ],
                style=("background-color: yellow; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
                )
    )
    # m.add_child(geo)
    geo.add_to(m)


    m.save('C:/Users/FilipeAfonso/Documents/ESG/map'+str(dept)+'.html')
    # m

    return


dept=75
jeu=0.010


#!!!!!!!!!!!!!!!!!!!!!!!!! ils nous manquent des batiments

# on récupère dans unofi les batiments du departement en paramètre et on les retrouve dans les bâtiments de la bnb
#à travers la table jointure df_ub

#on récupère dans unofi les batiments du departement
temp=df_unofi.T.apply(lambda x: True if str(eval(x['form_ADRESSE'])['departement']['numero'])==str(dept) else False)
unofi_actif=df_unofi.loc[temp[temp==True].index]

# on les retrouve la table jointure df_ub pour ensuite récupère les id bnb 
temp=df_ub.T.apply(lambda x: True if x['id_unofi'] in unofi_actif['code'].to_list() else False)
ub_actif=df_ub.loc[temp[temp==True].index]
print(ub_actif)

#on retrouve alors  les clients unofi dans les bâtiments de la bnb
temp=df_batiment.T.apply(lambda x: True if x['etaban202111_id'] in ub_actif['id_bnb'].to_list() else False)
bat_actif=df_batiment.loc[temp[temp==True].index]

build_map_light(bat_actif, df_batiment,jeu)

# bat_actif=df_batiment[df_batiment['etaban202111_id']==ub_actif.iloc[0][1]]

# for index,row in df_unofi.iterrows():
#     ub_actif=df_ub[df_ub['id_unofi']==row['code']]
#     if(len(ub_actif)>0 and str(eval(row['form_ADRESSE'])['departement']['numero'])==str(dept)):
#         commune=str(eval(row['form_ADRESSE'])['ville']['nom'])
#         print(commune)
#         bat_actif=df_batiment[df_batiment['etaban202111_id']==ub_actif.iloc[0][1]]
#         if(len(bat_actif)>0) :
#             build_map_light(row,bat_actif, df_batiment,jeu)
#             listeok = open('C:/Users/FilipeAfonso/Documents/ESG/liste_unofi.csv', "a")
#             print(row['code']+";"+eval(row['assetData.ADDRESS'])[0].get('value'),file=listeok)
#             listeok.close()
#             # sys.exit(0)

  id_unofi            id_bnb
0      75A  75117_3931_00007
1      75J  75109_8673_00081
2      75K  75108_5190_00011
3      75L  75116_1074_00050
4      75O  75114_2236_00034
5      75Q  75117_6242_00012
6      75P  75108_0179_00005
7     75MC  75110_7043_00026
8     75MB  75110_7043_00030
2.3115561919218557 48.87464899386285


In [ ]:
def set_couleurs():
    couleurs=['red','darkgreen','yellow','blue','darkkhaki',\
        'darkorange','peru','darkcyan' ]    
    return couleurs

def get_highlight(couleur):
    # style={'orange':'red','green':'darkgreen','yellow':'gold','skyblue':'blue','khaki':'darkkhaki',\
        # 'bisque':'darkorange','linen':'peru','darkcyan':'cyan' }
    style={'red':'orange','darkgreen':'green','yellow':'gold','blue':'skyblue','darkkhaki':'khaki',\
        'darkorange':'bisque','peru':'linen','darkcyan':'cyan' }
    return style[couleur]


def viz_batiment(vue,bat,couleur):
    

    geo_j = bat.to_json()
    geo_style_function = lambda x: {'color' :  couleur,'opacity' : 0.50,'weight' : 2,}
    geo_highlight_function = lambda x: {'color': get_highlight(couleur), 'opacity' : 0.9,'weight': 4,'dashArray' : '3, 6'}

    # main display function using the two previous functions
    geo = folium.features.GeoJson(
    data=geo_j,
    name = 'geo',
    control = True,
    style_function = geo_style_function, 
    highlight_function = geo_highlight_function,

    # the tooltip is where the info display happens
    # using "folium.features.GeoJsonTooltip" function instead of basic text tooltip
            tooltip=folium.features.GeoJsonTooltip(
                fields=[
                    'bnb_id',
                    vue, 
                ],
                aliases=[
                    "Identifiant: ",
                    "Filtre: ",
                ],
                style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
                )
    )

    return geo

def popup_batiment(bat_actif,clients,geo):
    next=0
    for _, r in bat_actif.iterrows(): 
        #folium.Marker((r.noyau.y,r.noyau.x),icon=folium.Icon(color='green')).add_to(geo)
        try:
            html=f"""
            <h5>{'Identifiant Stonal : ' + clients['code'].iloc[next]+" : "+eval(clients['assetData.ADDRESS'].iloc[next])[0].get('value')}</h5>
            <h5> {"identifiant BNB : "+r['bnb_id']}</h5>
            """
            iframe = folium.IFrame(html=html, width=600, height=100)
            popup = folium.Popup(iframe, max_width=600)
            # popup.add_to(geo)

            #folium.Marker((r.noyau.y,r.noyau.x), popup=clients['code'].iloc[next]+","+eval(clients['assetData.ADDRESS'].iloc[next])[0].get('value')),icon=folium.Icon(color='green')).add_to(geo)
            folium.Marker((r.noyau.y,r.noyau.x), popup=popup,icon=folium.Icon(color='green')).add_to(geo)
        except:
            # sys.exit(0)
            continue
        next=next+1
        
    return geo


def display_vue(vue,bat_actif,df_batiment,clients,jeu=0.015):
    couleurs=set_couleurs()

    xactif=float(bat_actif.iloc[0].noyau.x)
    yactif=float(bat_actif.iloc[0].noyau.y)
    print(xactif, yactif)
    m = folium.Map(location=(yactif,xactif), zoom_start=17, tiles='CartoDB positron')

    df_reduit=df_batiment[(df_batiment.noyau.x.between(xactif-jeu, xactif+jeu)) & (df_batiment.noyau.y.between(yactif-jeu, yactif+jeu))]
    df_reduit=df_reduit[['bnb_id','geometry',vue]]
    df_reduit=df_reduit[df_reduit.isnull().any(axis=1)==False]

    i=0
    print(df_batiment[vue].unique)
    for p in df_reduit[vue].unique():
        df_p=df_reduit[df_reduit[vue]==p]
        geo=viz_batiment(vue,df_p,couleurs[i])
        i=i+1
        if(i>7): i=0
        geo.add_to(m)
    
    geo=popup_batiment(bat_actif,clients,geo)

    
        
    m.save('C:/Users/FilipeAfonso/Documents/ESG/map'+str(dept)+'.html')

In [ ]:
# on récupère dans unofi les batiments du departement en paramètre et on les retrouve dans les bâtiments de la bnb
#à travers la table jointure df_ub

#on récupère dans unofi les batiments du departement
temp=df_unofi.T.apply(lambda x: True if str(eval(x['form_ADRESSE'])['departement']['numero'])==str(dept) else False)
unofi_actif=df_unofi.loc[temp[temp==True].index]

# on les retrouve la table jointure df_ub pour ensuite récupère les id bnb 
temp=df_ub.T.apply(lambda x: True if x['id_unofi'] in unofi_actif['code'].to_list() else False)
ub_actif=df_ub.loc[temp[temp==True].index]
print(ub_actif)

#on retrouve alors  les clients unofi dans les bâtiments de la bnb
temp=df_batiment.T.apply(lambda x: True if x['etaban202111_id'] in ub_actif['id_bnb'].to_list() else False)
bat_actif=df_batiment.loc[temp[temp==True].index]

# couleurs=['orange','green','yellow','skyblue','khaki','bisque','linen','darkcyan']

jeu=0.015


    

  id_unofi            id_bnb
0      75A  75117_3931_00007
1      75J  75109_8673_00081
2      75K  75108_5190_00011
3      75L  75116_1074_00050
4      75O  75114_2236_00034
5      75Q  75117_6242_00012
6      75P  75108_0179_00005
7     75MC  75110_7043_00026
8     75MB  75110_7043_00030


In [ ]:
vue='cerffo2020_mat_toit_txt'
vue='cerffo2020_mat_mur_txt'


display_vue(vue,bat_actif,df_batiment,unofi_actif,jeu)


2.3115561919218557 48.87464899386285
<bound method Series.unique of 0          BRIQUES - PIERRE
1                      None
2        BRIQUES - MEULIERE
3                    PIERRE
4                    PIERRE
                ...        
72346    BRIQUES - MEULIERE
72347    BRIQUES - MEULIERE
72348                  None
72349                PIERRE
72350    BRIQUES - MEULIERE
Name: cerffo2020_mat_mur_txt, Length: 72351, dtype: object>
